In [1]:
import numpy as np
import pandas as pd
from flask import Flask, abort, jsonify, request
import pickle
from prakash_helper import convert_json_DF,raw_final_DF
from sklearn.ensemble import GradientBoostingClassifier


with open('final_model.pkl', 'rb') as f:
        classifier = pickle.load(f)
DF_default = pd.read_pickle('Default_DF.pkl')

In [4]:
%ls 

Final_model.pkl    __pycache__/       default_DF.pkl
Untitled.ipynb     app.py             prakash_helper.py


In [5]:
pv=pd.read_csv('../prakash.csv')
ll=pv.to_json(orient='records')

In [21]:
pv

,goal,country,staff_pick,backers_count,usd_pledged,category,name_len_clean,blurb_len_clean,deadline_weekday,created_at_weekday,launched_at_weekday,create_to_launch_days,launch_to_deadline_days,LaunchedTuesday,DeadlineWeekend
0,1500,US,False,0,0,Academic,9,16,Friday,Saturday,Wednesday,17,36,0,0


In [23]:
ll

'[{"goal":1500,"country":"US","staff_pick":false,"backers_count":0,"usd_pledged":0,"category":"Academic","name_len_clean":9,"blurb_len_clean":16,"deadline_weekday":"Friday","created_at_weekday":"Saturday","launched_at_weekday":"Wednesday","create_to_launch_days":17,"launch_to_deadline_days":36,"LaunchedTuesday":0,"DeadlineWeekend":0}]'

In [10]:
def prepare_data(df):
    import numpy as np
    from sklearn.preprocessing import FunctionTransformer
    sc = FunctionTransformer(np.log1p)
    X=df[['goal']]
    X= sc.transform(X)
    df[['goal']]=X
    df = pd.get_dummies(df, columns=['country'])
    df = pd.get_dummies(df, columns=['category'])
    df = pd.get_dummies(df, columns=['deadline_weekday'])
    df = pd.get_dummies(df, columns=['created_at_weekday'])
    df = pd.get_dummies(df, columns=['launched_at_weekday'])
    return df

In [24]:
data=prepare_data(gg)

In [25]:
data

,DeadlineWeekend,LaunchedTuesday,backers_count,blurb_len_clean,create_to_launch_days,goal,launch_to_deadline_days,name_len_clean,staff_pick,usd_pledged,country_US,category_Academic,deadline_weekday_Friday,created_at_weekday_Saturday,launched_at_weekday_Wednesday
0,0,0,0,16,17,1.137168,36,9,False,0,1,1,1,1,1


In [26]:
DF_default.update(data)

In [27]:
DF_default["staff_pick"]=DF_default["staff_pick"].astype(int)

In [28]:
def do_the_computation():
    classifier,DF_default=load_pickle_file()
    # Assume that user has used to script to convert csv to json file and data.json file exist.
    X_test_raw = convert_json_DF('data.json')
    X_test = raw_final_DF(X_test_raw,DF_default)
    predict_me = classifier.predict(X_test)
    #print ('% Probability of sucess',100*classifier.predict_proba(X_test)[0][1])
    output = ('% Probability of sucess {}'.format(100*classifier.predict_proba(X_test)[0][1]))
    return output

In [35]:
classifier.predict(DF_default)

array([0])

In [36]:
DF_default

,goal,staff_pick,backers_count,name_len_clean,blurb_len_clean,create_to_launch_days,launch_to_deadline_days,LaunchedTuesday,DeadlineWeekend,country_AT,...,created_at_weekday_Thursday,created_at_weekday_Tuesday,created_at_weekday_Wednesday,launched_at_weekday_Friday,launched_at_weekday_Monday,launched_at_weekday_Saturday,launched_at_weekday_Sunday,launched_at_weekday_Thursday,launched_at_weekday_Tuesday,launched_at_weekday_Wednesday
0,1.137168,0,0.0,9.0,16.0,17.0,36.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [37]:
output = ('% Probability of sucess {}'.format(100*classifier.predict_proba(DF_default)[0][1]))

In [38]:
print(output)

% Probability of sucess 7.931023586284401
